In [1]:
!pip install python-docx
!python -m spacy download fr_core_news_md

2023-07-09 17:32:26.225266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 17:32:29.872573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 13.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import requests
import docx
import re
from google.colab import files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def get_word_definitions(word):
    # URL de l'API Wikimedia REST
    url = f"https://fr.wikipedia.org/api/rest_v1/page/summary/{word}"

    try:
        # Envoi de la requête HTTP GET
        response = requests.get(url)
        response.raise_for_status()  # Lever une exception si la requête n'a pas réussi

        data = response.json()

        if "extract" in data:
            # Filtrer les noms propres en excluant les paragraphes en majuscules
            definitions = [p.strip() for p in data["extract"].split("\n") if not p.isupper()]
            return definitions

    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")

    return []

def get_definitions_from_text(text):
    # Suppression des apostrophes
    text = re.sub(r"\b'\b", "", text)

    # Suppression des parenthèses, des virgules, des points et des guillemets
    text = re.sub(r"[\(\),.\"'-:;+*/]", "", text)

    # Séparation du texte en mots individuels
    words = word_tokenize(text, language="french")

    # Élimination des stopwords
    stop_words = set(stopwords.words("french"))
    words = [word for word in words if word.lower() not in stop_words]

    # Élimination des chiffres
    words = [word for word in words if not word.isdigit()]

    # Chargement du modèle français de SpaCy
    nlp = spacy.load("fr_core_news_md")

    # Analyse du texte avec SpaCy
    doc = nlp(" ".join(words))

    # Récupération des définitions pour chaque mot, en excluant les noms propres
    word_definitions = {}
    for token in doc:
        if not token.ent_type_ and token.pos_ != "PROPN":  # Exclure les noms propres et les entités nommées
            word = token.text
            definitions = get_word_definitions(word)
            if definitions:
                word_definitions[word] = definitions

    return word_definitions

In [4]:
# Télécharger le fichier .doc depuis l'ordinateur local vers Colab
uploaded = files.upload()

# Récupérer le nom du fichier téléchargé
file_name = next(iter(uploaded))

# Lire le contenu du fichier .doc
doc = docx.Document(file_name)

# Extraire le texte du document
text = " ".join([paragraph.text for paragraph in doc.paragraphs])


Saving IA.docx to IA (2).docx


In [5]:
definitions = get_definitions_from_text(text)

# Créer un nouveau document .doc
new_doc = docx.Document()

if definitions:
    new_doc.add_heading("Définitions des mots dans le texte (hors noms propres, stopwords, chiffres et ponctuation)", level=1)
    for word, word_defs in definitions.items():
        new_doc.add_heading(f"Mot : {word}", level=2)
        for definition in word_defs:
            new_doc.add_paragraph(definition)
else:
    new_doc.add_paragraph("Aucune définition trouvée pour les mots dans le texte.")

Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Linformatique
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Concentre
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Capables
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Deffectuer
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/N%C3%A9cessitent
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Lintelligence
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/summary/Permettre
Une erreur s'est produite : 404 Client Error: Not Found for url: https://fr.wikipedia.org/api/rest_v1/page/sum

In [6]:
# Créer un nouveau document .doc
new_doc = docx.Document()

if definitions:
    new_doc.add_heading("Définitions des mots dans le texte (hors noms propres et stopwords)", level=1)
    for word, word_defs in definitions.items():
        new_doc.add_heading(f"Mot : {word}", level=2)
        for definition in word_defs:
            new_doc.add_paragraph(definition)
else:
    new_doc.add_paragraph("Aucune définition trouvée pour les mots dans le texte.")

# Enregistrer le document .doc
output_file = "Wiki.doc"
new_doc.save(output_file)

# Télécharger le fichier .doc
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>